In [10]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.densenet import DenseNet201

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from utils import load_data, load_test_data
from utils import num_classes, epochs, batch_size
import keras.backend as K

# DenseNet201 Fine Tuning

In [11]:
X_train, y_train, X_valid, y_valid = load_data(Gray2RGB=True, mean_proc='DenseNet', img_size=224, test_size=0.1)

In [12]:
model_name = 'DenseNet201-Fine-Tune'

img_rows, img_cols, img_channel = 224, 224, 3
base_model = DenseNet201(weights='imagenet', include_top=False,
                         input_shape=(img_rows, img_cols, img_channel))

for layer in base_model.layers:
    #layer.trainable = False
    if hasattr(layer, 'moving_mean') and hasattr(layer, 'moving_variance'):
        layer.trainable = True
        #K.eval(K.update(layer.moving_mean, K.zeros_like(layer.moving_mean)))
        #K.eval(K.update(layer.moving_variance, K.zeros_like(layer.moving_variance)))
    else:
        layer.trainable = False
# or if we want to set the first 20 layers of the network to be non-trainable
#for layer in model.layers[:20]:
#    layer.trainable=False
#for layer in model.layers[20:]:
#    layer.trainable=True

x = base_model.output
x = GlobalAveragePooling2D(data_format='channels_last')(x)
x = Dense(128,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
#x = Dropout(0.5)(x)
#x = BatchNormalization(epsilon=eps, axis=concat_axis, name='conv1_bn')(x)
x = Dense(128,activation='relu')(x) #dense layer 2
#x = BatchNormalization(epsilon=eps, axis=concat_axis, name='conv1_bn')(x)
#x = Dropout(0.5)(x)

predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [ ]:
from keras.callbacks import ReduceLROnPlateau

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')


model_path = './saved_models/DensetNet_0513001.h5'.format(model_name)

best_model = []
count = 0

while count < 10:
    model_path = './saved_models/DensetNet_0513001_{}.h5'.format(str(count))
    
    optimizer = keras.optimizers.Adam(lr=10e-4,beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)

    checkpoint = ModelCheckpoint(model_path, monitor='val_acc', save_best_only=True, verbose=0)

    learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.000001, cooldown=2)

    earlystop = EarlyStopping(monitor='val_acc', patience=16, verbose=1)

    lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                                           factor=0.1,
                                           patience=3,
                                           min_lr=0.5e-6)
    
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer, metrics=['accuracy'])

    batch_size = 16
    
    model_history = model.fit_generator(datagen.flow(X_train, y_train, batch_size = batch_size),
                                        epochs = epochs,
                                        validation_data = (X_valid, y_valid),
                                        callbacks = [checkpoint,earlystop])
    # 重載當下最佳 Model (val_loss最低)
    model = load_model(model_path)

    scores = model.evaluate(X_valid, y_valid, verbose=1)
    print('Validation loss:', scores[0])
    print('Validation accuracy:', scores[1])
    
    # 儲存最佳 Model (Test acc 超過 0.55)
    #if test_accuracy > 0.65:
    #    print("Save best model(test_acc:%4f) ..."%(test_accuracy))
    #    best_model.append(model)
    count += 1


Epoch 1/400
31/31 [==============================] - 942s 30s/step - loss: 0.0990 - acc: 0.9657 - val_loss: 2.0171 - val_acc: 0.6296
Epoch 2/400
31/31 [==============================] - 13s 407ms/step - loss: 0.1081 - acc: 0.9520 - val_loss: 1.8950 - val_acc: 0.6852
Epoch 3/400
31/31 [==============================] - 13s 409ms/step - loss: 0.0732 - acc: 0.9677 - val_loss: 1.0355 - val_acc: 0.7778
Epoch 4/400
31/31 [==============================] - 7s 236ms/step - loss: 0.0573 - acc: 0.9798 - val_loss: 1.2569 - val_acc: 0.7407
Epoch 5/400
31/31 [==============================] - 8s 244ms/step - loss: 0.0440 - acc: 0.9820 - val_loss: 1.0937 - val_acc: 0.7778
Epoch 6/400
31/31 [==============================] - 7s 233ms/step - loss: 0.0790 - acc: 0.9818 - val_loss: 1.5328 - val_acc: 0.6481
Epoch 7/400
31/31 [==============================] - 7s 238ms/step - loss: 0.0939 - acc: 0.9617 - val_loss: 1.3618 - val_acc: 0.6481
Epoch 8/400
31/31 [==============================] - 7s 235ms/step 

In [ ]:
training_loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

plt.plot(training_loss, label="training_loss")
plt.plot(val_loss, label="validation_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Learning Curve")
plt.legend(loc='best')
plt.show()

In [ ]:
training_acc = model_history.history['acc']
val_acc = model_history.history['val_acc']

plt.plot(training_acc, label="training_acc")
plt.plot(val_acc, label="validation_acc")
plt.xlabel("Epochs")
plt.ylabel("Acc")
plt.title("Learning Curve")
plt.legend(loc='best')
plt.show()

In [16]:
X_test, X_id = load_test_data(Gray2RGB=True, img_size=224, mean_proc='DenseNet')

model_path = './saved_models/DensetNet_0513001.h5'.format(model_name)
model = load_model(model_path)

#scores = model.evaluate(X_valid, y_valid, verbose=1)
#print('Validation loss:', scores[0])
#print('Validation accuracy:', scores[1])

y_test_pred_prob = model.predict(X_test)
y_test_pred = y_test_pred_prob.argmax(axis=-1)
y_test_pred_df = pd.DataFrame({'id': np.array(X_id), 'class':y_test_pred}).sort_values(by='id')
y_test_pred_df.to_csv('DensetNet_0513001.csv'.format(model_name), index=False)